###### excel -> csv 변환

In [0]:
import pandas as pd
import numpy as np
import os

In [0]:
pwd

'C:\\MarketCaster\\Industrial Index'

![대체 텍스트](https://)[링크 텍스트](https://)

In [0]:
# 년, 월, 일로 표기된 DATE를 처리하기 위한 function
# input 인자인 dataFrame은 AMD['DATE']와 같이 dataframe 내 컬럼을 명시해주어야 함
# 2019년 12월 4일 -> 2019-12-04
def date_preprocessing(dataFrame):
    i = 0
    for date in dataFrame:
        try:
            # 띄어쓰기 제거
            tmp1, tmp2, tmp3 = date.split(' ')
            # 연, 월, 일 조립
            tmp = tmp1[:4] + '-' + tmp2[:2] + '-' + tmp3[:2]
        except:
            tmp = date
        # reset
        dataFrame.loc[i] = pd.to_datetime(tmp)
        i = i + 1

In [0]:
def comma_replace(dataFrame):
    i = 0
    for element in dataFrame:
        try:
            element = element.replace(',', '')
            dataFrame.loc[i] = float(element)
        except:
            pass
        i += 1

In [0]:
# 거래량이 M(Milion) 단위로 표기된 VOLUME을 처리하기 위한 function
# input 인자인 dataFrame은 AMD['VOLUME']와 같이 dataframe 내 컬럼을 명시해주어야 함
# 10.00M -> 10000000
def volume_preprocessing(dataFrame):
    i = 0
    for volume in dataFrame:
        if type(volume) == str:
            if volume[-1] == 'M':
                volume = int(float(volume[:-1]) * 1000000)
            elif volume == '-':
                volume = 0
            dataFrame.loc[i] = int(volume)
        elif volume == np.nan:
            pass
        else:
            try:
                dataFrame.loc[i] = int(dataFrame.loc[i])
            except:
                pass
        i = i + 1

In [0]:
# 일별수익률에 %기호가 붙어있는 경우 사용
def change_preprocessing(dataFrame):
    i = 0
    for change in dataFrame:
        try:
            # %기호 제거
            if change[-1] == '%':
                change = change[:-1]
        except:
            pass
        try:
            dataFrame.loc[i] = float(change)
        except:
            print(type(change), ' ', i)
        i = i + 1

##### 시계열 데이터 from to 설정

In [0]:
# 사용할 데이터의 기간 설정
# date_from < 사용할 기간 < date_to
date_from = pd.to_datetime('2006-01-01')
date_to   = pd.to_datetime('2019-11-01')

# investing.com 데이터 중 비영업일이 영업일로 체크된 날짜 설정
investing_ignore_date = pd.to_datetime('2016-02-27')

##### Directory 위치 설정

In [0]:
# data file이 위치한 directory 설정
path = './Data/Currency/'
csv_path = './Data/CSV/'

# 최종 data name 설정
describe_file_name = 'describe_Currency.csv'

In [0]:
# directory 내 전체 file list 추출
file_list = os.listdir(path)

In [0]:
# csv file 추출
file_list_csv = [file for file in file_list if file.endswith(".csv")]

In [0]:
# xlsx file 추출
file_list_xlsx = [file for file in file_list if file.endswith(".xlsx")]

In [0]:
# 파일명을 저장하기 위한 list 선언
df_name_list = []
# dataFrame을 저장하기 위한 list 선언
df_list = []

In [0]:
# directory 내 csv file read
for file in file_list_csv:
    df_list.append(pd.read_csv(path+file, encoding='CP949'))
    df_name_list.append(file[:-4])

In [0]:
# directory 내 xlsx file read
for file in file_list_xlsx:
    df_list.append(pd.read_excel(path+file, sheet_name='Sheet1'))
    df_name_list.append(file[:-5])

In [0]:
# 컬럼의 구성이 다른 경우가 있는지 check
max_column_length = 0
i = 0
for df in df_list:
    if len(df.columns) > max_column_length:
        max_column_length = len(df.columns)
        df_columns = df.columns
    if list(df_columns) != list(df.columns):
        print(df_name_list[i])
        print('기준 컬럼 : ', list(df_columns))
        print('다른 컬럼 : ', list(df.columns))
        print('error')
        break
    i += 1

USD_KRW
기준 컬럼 :  ['DATE', 'OPEN', 'HIGH', 'LOW', 'CLOSE', 'VOLUME', 'CHANGE']
다른 컬럼 :  ['DATE', 'OPEN', 'HIGH', 'LOW', 'CLOSE', 'CHANGE']
error


In [0]:
# VOLUME 컬럼이 없는 dataFrame에 추가
i = 0
for df in df_list:
    if list(df_columns) != list(df.columns):
        df.insert(loc=5, column='VOLUME', value=0)
    i += 1

In [0]:
for df in df_list:
    for element in ['OPEN', 'HIGH', 'LOW', 'CLOSE']:
        comma_replace(df[element])
        df[element] = df[element].apply(pd.to_numeric)
# df['CHANGE'] = df['CHANGE'].apply(pd.to_numeric)

C:\Users\Seungwon\AppData\Local\Continuum\miniconda3\lib\site-packages\pandas\core\indexing.py:205: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


In [0]:
tmp_df_list = []
for df in df_list:
    # 날짜 데이터 일원화
    date_preprocessing(df['DATE'])
    # 거래량 데이터 일원화
    volume_preprocessing(df['VOLUME'])
    # 일별수익률 데이터 일원화
    change_preprocessing(df['CHANGE'])
    df['CHANGE'] = df['CHANGE'].apply(pd.to_numeric)
    
    # 데이터 기간 일원화
    tmp_df = df[df['DATE'] < date_to]
    
    # investing.com에서 영업일로 표기된 날짜에 대한 index 추출 및 행 삭제
    try:
        index = df[df['DATE'] == investing_ignore_date].index[0]
        tmp_df = tmp_df.drop(index, axis = 0)
        tmp_df_list.append(tmp_df)
    except:
        tmp_df_list.append(tmp_df)
    

-------

In [0]:
describe_df_list = pd.DataFrame(columns=df_columns[1:])
i = 0
for df in tmp_df_list:
    date_data = pd.DataFrame([[df_name_list[i], 'Date_From', df['DATE'].min(), 'Date_To', df['DATE'].max(), len(df['DATE'])]],
                             columns=df_columns[1:])
    describe_df_list = pd.concat([describe_df_list, date_data], axis = 0, sort=False)
    describe_df_list = pd.concat([describe_df_list, df.describe()], axis = 0, sort=False)
    i += 1

In [0]:
describe_df_list.to_csv(describe_file_name)

In [0]:
# csv file save
i = 0
for df in tmp_df_list:
    df.to_csv(csv_path+df_name_list[i]+'.csv', index=False)
    i += 1